[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eliewolfe/d-separation/blob/main/igraph_playground.ipynb)

In [39]:
import numpy as np
from igraph import *

array([[1, 2],
       [3, 4]])

In [113]:
#import pip
#pip.main(['install','--upgrade','python-igraph'])

In [114]:
#Here are some useful igraph commands:
#get_inclist: can be used to obtain a vertex's parents or children
#subcomponent: can be used to obtain a vertex's ancestors or descendants
#all_st_mincuts: find screen off sets
#all_minimal_st_separators()  find screen off sets, made not be useful output
#minimum_size_separators() not sure what the difference is with previous, only works on undirected

In [ ]:
#it may be much easier to use graph-tools https://graph-tool.skewed.de/static/doc/index.html, not on Windows
#or networkx

In [186]:
#Graph([(0,1), (0,2), (2,3), (3,4), (4,2), (2,5), (5,0), (6,3), (5,6)])

In [187]:
g=Graph.TupleList([('U','X'),('X','A'),('A','B'),('L','A'),('L','B')],directed=True);
reorder=g.topological_sorting('out')
reorder=np.argsort(reorder).tolist()
g=g.permute_vertices(reorder)
g.vs['name']

['U', 'L', 'X', 'A', 'B']

In [188]:
[e.tuple for e in g.es]

[(0, 2), (2, 3), (3, 4), (1, 3), (1, 4)]

In [189]:
parentslist=g.get_adjlist('in')
parentslist

[[], [], [0], [1, 2], [1, 3]]

In [190]:
ancestorstable=[g.subcomponent(i,'in') for i in g.vs]
ancestorstable

[[0], [1], [2, 0], [3, 1, 2, 0], [4, 1, 3, 2, 0]]

In [191]:
numbered_vertices=np.arange(g.vcount());
rootvsnot=g.indegree()
rootvsnot=np.array(rootvsnot,np.bool)
rootvsnot

array([False, False,  True,  True,  True])

In [192]:
nonroot_vertices=numbered_vertices[rootvsnot]
root_vertices=numbered_vertices[~rootvsnot]
latent_count=len(root_vertices)

In [193]:
root_structure=[np.intersect1d(anc,root_vertices) for anc in ancestorstable]
root_structure

[array([0]), array([1]), array([0]), array([0, 1]), array([0, 1])]

In [194]:
determinism_concerns=np.array([[l,idx] for idx,setpair in enumerate(zip(root_structure,parentslist)) for l in np.setdiff1d(*setpair) if l!=idx])
determinism_concerns

array([[0, 3],
       [0, 4]])

In [195]:
rootdecendantstable=[g.subcomponent(i,'out') for i in root_vertices]
rootdecendantstable

[[0, 2, 3, 4], [1, 3, 4]]

In [196]:
[(concern[0],np.intersect1d(rootdecendantstable[concern[0]],parentslist[concern[1]]).tolist().append(concern[1])) for concern in determinism_concerns]

[(0, None), (0, None)]

In [197]:
g.all_minimal_st_separators()

[[2], [3]]

In [198]:
def FindScreenSet(concern):
    L=concern[0];
    V=concern[1];
    upcone_ofL=np.array(rootdecendantstable[L])
    downcone_ofV=np.array(parentslist[V])
    screeningset=np.intersect1d(upcone_ofL,downcone_ofV).tolist()
    screeningset.append(V)
    return screeningset
[(concern[0],FindScreenSet(concern)) for concern in determinism_concerns]

[(0, [2, 3]), (0, [3, 4])]